# The control of human psycho-affective stability using differential equations

In [24]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Functions 

### Define the system

In [25]:
def romeo_juliet_system(t, variables, a, b, c, d):
    """
    Defines the linear system modeling Romeo and Juliet's emotions.
    dx/dt = a*x + b*y
    dy/dt = c*x + d*y
    """
    x, y = variables
    dxdt = a * x + b * y
    dydt = c * x + d * y
    return [dxdt, dydt]

### Equilibrium and stability computations

In [26]:
def analyze_stability(a, b, c, d):
    """
    Computes determinant, Jacobian, eigenvalues and interprets stability.
    """
    J = np.array([[a, b], [c, d]])
    delta = a * d - b * c
    eigenvalues, _ = np.linalg.eig(J)

    print(f"Jacobian:\n{J}")
    print(f"Determinant Δ = {delta:.2f}")
    print(f"Eigenvalues = {eigenvalues}\n")

    if np.isreal(eigenvalues).all():  # Real eigenvalues
        if np.sign(eigenvalues[0]) == np.sign(eigenvalues[1]):
            if np.all(eigenvalues < 0):
                return "Stable node"
            elif np.all(eigenvalues > 0):
                return "Unstable node"
        else:
            return "Saddle point (unstable)"
    else:
        real_part = np.real(eigenvalues[0])
        if real_part < 0:
            return "Stable spiral"
        elif real_part > 0:
            return "Unstable spiral"
        else:
            return "Center (neutral oscillations)"

### Function to modelise/simulate

In [27]:
def simulate_system(a, b, c, d, x0=0.5, y0=0.5):
    """
    Solves the system numerically over t ∈ [0, 70].
    """
    t_span = (0, 70)
    t_eval = np.linspace(*t_span, 1000)
    sol = solve_ivp(romeo_juliet_system, t_span, [x0, y0], args=(a, b, c, d), t_eval=t_eval)
    return sol

### The function to plot

In [28]:
def plot_evolution(sol, title):
    """
    Plots the emotional evolution of Romeo and Juliet over time.
    """
    plt.figure(figsize=(7, 4))
    plt.plot(sol.t, sol.y[0], label="Romeo (x)", color='lightblue')
    plt.plot(sol.t, sol.y[1], label="Juliet (y)", color='darkpink')

    plt.xlabel("Time")
    plt.ylabel("Emotions")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### THE function

In [29]:
def run_scenario(name, a, b, c, d, x0=0.5, y0=0.5):
    """
    Runs one full scenario: stability + simulation + plot.
    """
    print("\n----------------------------------------")
    print(f"SCENARIO: {name}")
    print(f"Parameters: a={a}, b={b}, c={c}, d={d}")
    
    stability = analyze_stability(a, b, c, d)
    print(f"→ Stability type: {stability}")
    
    sol = simulate_system(a, b, c, d, x0, y0)
    print(f"Final state: x={sol.y[0, -1]:.3f}, y={sol.y[1, -1]:.3f}")
    
    plot_evolution(sol, f"{name} – {stability}")

How to use it : copy and paste the following code :


In [ ]:
#scenarios = {
#    "scenario 1": (0, 0, 0, 0),
#    "scenario 2": (0, 0, 0, 0),
#    } # Add scenarios as needed

# Run them all
#for name, params in scenarios.items():
#    run_scenario(name, *params, x0=0.1, y0=-0.1)

# The tests

In [ ]:
scenarios = {
    "scenario 1": (0, 0, 0, 0),
    "scenario 2": (0, 0, 0, 0),
    }

# Run them all
for name, params in scenarios.items():
    run_scenario(name, *params, x0=0.1, y0=-0.1)